# Map georeferencing notebook - Python

In [25]:
import pandas as pd
from geopy.geocoders import Nominatim
import os
import folium
# from folium import Choropleth, Circle, Marker
# from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

import datetime
import locale

from matplotlib import colors as mcolors

import copy

In [26]:
# url = "https://raw.githubusercontent.com/AngelusGi/UniMolFinalExam/main/Dataset/Original_CSV/Angelo/trackLog-2019-dic-23_13-53-06.csv"
# file_name = "trackLog-2019-dic-23_13-53-06.csv"

In [27]:
# urllib.request.urlretrieve(url, filename=file_name)

Questo metodo elimina dal corrente dataframe tutte le colonne, eccezion fatta per latitudine e longitudine

In [28]:
def clean_header(file_to_process):

    dataTypeDict = {"GPS Time":str ,"Device Time":str ,"Longitude":float, "Latitude":float, "GPS Speed (Meters/second)":float, "Altitude":float, "G(x)":float, "G(y)":float, "G(z)":float, "Kilometers Per Litre(Instant)(kpl)": float, "Litres Per 100 Kilometer(Instant)(l/100km)":float, "Actual engine % torque(%)":float, "Trip Distance(km)":float, "Engine RPM(rpm)":float, "GPS Accuracy(m)":float, "GPS Altitude(m)":float, "GPS Latitude(°)":float, "GPS Longitude(°)":float, "Acceleration Sensor(X axis)(g)":float, "Acceleration Sensor(Y axis)(g)":float, "Acceleration Sensor(Z axis)(g)":float, "Acceleration Sensor(Total)(g)":float, "Run time since engine start(s)":int, "Driver":str}

    df = pd.read_csv(file_to_process, dtype=dataTypeDict)
    # keeps_only_latidue_longitude = [ "GPS Time","Device Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]
    keeps_only_latidue_longitude_time = [ "GPS Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]

    # gps_long_lat = df.drop(columns=keeps_only_latidue_longitude)    
    # return gps_long_lat

    gps_long_lat_time = df.drop(columns=keeps_only_latidue_longitude_time)
    return gps_long_lat_time

In [29]:
def add_csv_to_df(file_to_process):
    return pd.read_csv(file_to_process)

codice per la gestione delle time stanp nel csv

In [30]:
def ParseGpsTime(gps_value:str):
    local_setting_gps_time = 'en_US'
    locale.setlocale(locale.LC_ALL, local_setting_gps_time)

    gps_format = '%a %b %d %H:%M:%S %z %Y'
    # gps_test = "Mon Dec 30 09:16:50 +0100 2019"

    date_time_gps = datetime.datetime.strptime(gps_value, gps_format)

    # print("GPS:")
    # print('Date:', date_time_gps.date())
    # print('Time:', date_time_gps.time())
    # print('Date-time:', date_time_gps)

    return date_time_gps

In [31]:
def ParseDeviceTime(device_value:str):
    local_setting_device_time = 'it_IT'
    locale.setlocale(locale.LC_ALL, local_setting_device_time)

    device_format = '%d-%b-%Y %H:%M:%S.%f'
    # device_time = "30-dic-2019 09:16:52.347"

    date_time_device = datetime.datetime.strptime(device_value, device_format)

    # print("Device:")
    # print('Date:', date_time_device.date())
    # print('Time:', date_time_device.time())
    # print('Date-time:', date_time_device)

    return date_time_device

prede tutti i file nella cartella e li trasforma in un dizionario di dataframe

In [32]:
basepath = os.environ['USERPROFILE']

local_user = "angel"
vm_user = "vmuser"

if local_user in basepath:
    githubpath = r"\GitHub\UniMolFinalExam"
    current_env = local_user
elif vm_user in basepath:
    githubpath = r"\Desktop\GitHub\UniMolFinalExam"
    current_env = vm_user

In [33]:
# csv_path = r"{}{}\Notebooks\Python\TestSet".format(basepath,githubpath)
csv_path = r"{}{}\Dataset\Final".format(basepath,githubpath)

In [34]:
coordinates_data = dict()
full_data = dict()

In [35]:
for directory, subdir_list, file_list in os.walk(csv_path):
    for file_name in file_list:
        print('File:', file_name)
        path_to_read_csv = csv_path + "\\" + file_name
        print(path_to_read_csv)

        temp_dataframe = clean_header(path_to_read_csv)
        temp_file_name_ext = file_name.replace('.csv','')
        final_file_name = temp_file_name_ext.replace('trackLog-','')
        coordinates_data[final_file_name] = temp_dataframe

        full_data[final_file_name] = add_csv_to_df(path_to_read_csv)
        print()
    print()

File: A.csv
C:\Users\angel\GitHub\UniMolFinalExam\Dataset\Final\A.csv

File: B.csv
C:\Users\angel\GitHub\UniMolFinalExam\Dataset\Final\B.csv




crea un dizionario contenete solo le coordinate di latitudine e longitudine

In [36]:
coordinates_data

{'A':                     Device Time  Longitude   Latitude
 0      23-dic-2019 13:53:09.763  13.931077  42.048168
 1      23-dic-2019 13:53:12.432  13.931068  42.048161
 2      23-dic-2019 13:53:15.081  13.931075  42.048154
 3      23-dic-2019 13:53:17.284  13.931077  42.048154
 4      23-dic-2019 13:53:19.504  13.931077  42.048155
 ...                         ...        ...        ...
 74438  01-gen-2020 13:19:56.110  13.931075  42.048150
 74439  01-gen-2020 13:19:57.318  13.931076  42.048151
 74440  01-gen-2020 13:19:58.529  13.931075  42.048142
 74441  01-gen-2020 13:19:59.743  13.931065  42.048130
 74442  01-gen-2020 13:20:00.949  13.931066  42.048119
 
 [74443 rows x 3 columns],
 'B':                      Device Time  Longitude   Latitude
 0       07-dic-2019 18:55:34.543  13.926052  42.050242
 1       07-dic-2019 18:55:35.276  13.926052  42.050242
 2       07-dic-2019 18:55:35.430  13.926052  42.050242
 3       07-dic-2019 18:55:38.522  13.926060  42.050236
 4       07-dic-2019 

In [37]:
drivers_list = ["A", "B"]

for driver in drivers_list:
    print("Driver -> {}".format(driver))

    print("Info ->")
    print()
    print(coordinates_data[driver].info())
    print()
    
    print("Head ->")
    print()
    print(coordinates_data[driver].head())
    print()
    print("##########################################")
    print()

Driver -> A
Info ->

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74443 entries, 0 to 74442
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Device Time  74443 non-null  object 
 1   Longitude    74443 non-null  float64
 2   Latitude     74443 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.7+ MB
None

Head ->

                Device Time  Longitude   Latitude
0  23-dic-2019 13:53:09.763  13.931077  42.048168
1  23-dic-2019 13:53:12.432  13.931068  42.048161
2  23-dic-2019 13:53:15.081  13.931075  42.048154
3  23-dic-2019 13:53:17.284  13.931077  42.048154
4  23-dic-2019 13:53:19.504  13.931077  42.048155

##########################################

Driver -> B
Info ->

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188795 entries, 0 to 188794
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Device Time  188795 non-nul

estrae un singolo dataframe

In [38]:
gps_long_lat_B = coordinates_data["B"]
gps_long_lat_A = coordinates_data["A"]

device_timeA = ParseDeviceTime(gps_long_lat_A["Device Time"][0])
device_timeB = ParseDeviceTime(gps_long_lat_B["Device Time"][0])

In [39]:
print("A : {}".format(device_timeA))
print("B : {}".format(device_timeB))
device_timeA.date() == device_timeB.date()

A : 2019-12-23 13:53:09.763000
B : 2019-12-07 18:55:34.543000


False

In [40]:
first_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][0])
second_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][1])
third_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][17248])
fourth_device_time = ParseDeviceTime(gps_long_lat_A["Device Time"][17249])

In [41]:
# test Times from the same day
print("A 1st : {}".format(first_device_time))
print("A 2nd : {}".format(second_device_time))
first_device_time.date() == second_device_time.date()

A 1st : 2019-12-23 13:53:09.763000
A 2nd : 2019-12-23 13:53:12.432000


True

In [42]:
# test Times from different days
print("A 1st : {}".format(first_device_time))
print("A 3nd : {}".format(third_device_time))
first_device_time.date() == third_device_time.date()

A 1st : 2019-12-23 13:53:09.763000
A 3nd : 2019-12-30 23:58:26.738000


False

In [43]:
# test Times from different days (close days/time)
print("A 3nd : {}".format(third_device_time))
print("A 4nd : {}".format(fourth_device_time))
third_device_time.date() == fourth_device_time.date()

A 3nd : 2019-12-30 23:58:26.738000
A 4nd : 2019-12-31 00:01:41.749000


False

ottenere un indirizzo a partire dalle coordinate

In [44]:
geolocator = Nominatim(user_agent="UniMolFinalExam")

In [45]:
place_to_find = "Italy"
geo_cordinates = geolocator.geocode(place_to_find)
geo_cordinates

Location(Italia, (42.6384261, 12.674297, 0.0))

In [46]:
map_zoom = 6
italy_map_A = folium.Map(location=[geo_cordinates.latitude,geo_cordinates.longitude], zoom_start=map_zoom)
italy_map_B = folium.Map(location=[geo_cordinates.latitude,geo_cordinates.longitude], zoom_start=map_zoom)

In [47]:
italy_map_A

aggiunte i punti sulla mappa, passaggio di doppio oggetto "map" perché python non digerisce bene la copia di oggetti ricorsivi

In [48]:
def Add_Points_To_Map(users_list, dataframe, map_A, map_B):
    map_dictionary = dict()

    color_code = 1
    memo_copy = {}

    for user in users_list:
        current_map = None
        color_list = None
        color_list = copy.deepcopy(mcolors.CSS4_COLORS)
        # color_list = copy.deepcopy(mcolors.CSS4_COLORS, memo=memo_copy)

        if (user == "A"):
            current_map = map_A
        else:
            current_map = map_B
        
        # color_list = {'beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray',
        #               'green', 'lightblue', 'lightgray', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white'}

        current_color = color_list.popitem()

        for index in range(len(dataframe[user])):

            current_row = dataframe[user].iloc[index]
            
            actual_device_time = ParseDeviceTime(
                current_row["Device Time"])

            if (index > 0):
                previous_row = dataframe[user].iloc[index-1]
                previous_device_time = ParseDeviceTime(
                    previous_row["Device Time"])

                if(previous_device_time.date() != actual_device_time.date()):
                    # se giorni diversi cambia colore cambia colore
                    current_color = color_list.popitem()

            folium.Circle(
                location = [current_row["Latitude"], current_row["Longitude"]],
                radius = 10,
                popup = "Driver: {} | {}".format(
                    user, actual_device_time.date()),
                color = current_color[color_code],
                fill = True,
                fill_color = current_color[color_code]
            ).add_to(current_map)

        map_dictionary[user] = current_map

    return map_dictionary

crea un dizionario di mappe {"driver":map}

In [49]:
driverMapDict = Add_Points_To_Map(users_list=drivers_list, dataframe=coordinates_data, map_A=italy_map_A, map_B=italy_map_B)

esporta la mappa prodotta in un file html

In [50]:
for driver in drivers_list:
    driverMapDict[driver].save("Driver{}_full_map.html".format(driver))

visualizzazione mappe esportate

In [51]:
driverMapDict["A"]

In [52]:
driverMapDict["B"]